In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from sklearn.metrics.pairwise import cosine_similarity
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances
from scipy.spatial.distance import correlation

In [2]:
# Load the pre-trained VGG19 model + higher level layers
base_model = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)


In [3]:
def load_yolo_model():
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    return net, classes

In [4]:
def detect_objects_yolo(img, net, classes):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    height, width, channels = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    return [(class_ids[i], boxes[i]) for i in range(len(boxes)) if i in indexes]


In [5]:
def extract_face(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
    
    if len(faces) == 0:
        return None
    
    x, y, w, h = max(faces, key=lambda rect: rect[2] * rect[3])
    face = img[y:y+h, x:x+w]
    return face

In [6]:
def extract_features(img_path, model, net, classes):
    face = extract_face(img_path)
    img = cv2.imread(img_path)
    if face is None:
        print(f"No face detected in image: {img_path}")
        face_features = np.zeros((4096,))
    else:
        face = cv2.resize(face, (224, 224))
        img_data = image.img_to_array(face)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        face_features = model.predict(img_data).flatten()
    
    objects = detect_objects_yolo(img, net, classes)
    object_features = []
    for class_id, (x, y, w, h) in objects:
        obj = img[y:y+h, x:x+w]
        if obj.size == 0:
            continue
        obj = cv2.resize(obj, (224, 224))
        obj_data = image.img_to_array(obj)
        obj_data = np.expand_dims(obj_data, axis=0)
        obj_data = preprocess_input(obj_data)
        obj_features = model.predict(obj_data).flatten()
        object_features.append(obj_features)
    
    if len(object_features) == 0:
        object_features = np.zeros((4096,))
    else:
        object_features = np.mean(object_features, axis=0)
    
    combined_features = np.concatenate([face_features, object_features])
    return combined_features

In [7]:
def load_dataset(dataset_path, model, net, classes):
    features = []
    image_paths = []
    labels = []
    for actor in os.listdir(dataset_path):
        actor_dir = os.path.join(dataset_path, actor)
        if os.path.isdir(actor_dir):
            for img_file in os.listdir(actor_dir):
                img_path = os.path.join(actor_dir, img_file)
                if img_path.endswith(('jpg', 'jpeg', 'png')):  # Check for valid image files
                    embedding = extract_features(img_path, model, net, classes)
                    if embedding is not None:
                        features.append(embedding)
                        labels.append(actor)
                        image_paths.append(img_path)
    return np.array(features), image_paths, labels

In [9]:
# Load the YOLO model and classes
net, classes = load_yolo_model()

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:217: error: (-212:Parsing error) Failed to parse NetParameter file: yolov3.weights in function 'cv::dnn::dnn4_v20231225::readNetFromDarknet'


In [ ]:
dataset_path = "D:\\Programming\\Project\\dataset_5"
# Load dataset and extract features
dataset_features, dataset_image_paths, labels = load_dataset_2(dataset_path, model)


In [ ]:
given_img_path = 'D:\\Programming\\Project\\pexels-photo-2379005.jpeg'
input_img = image.load_img(given_img_path, target_size=(224, 224))
input_img = image.img_to_array(input_img) / 255.0
plt.imshow(input_img)

In [ ]:
# Cosine similarity (METHOD 1)
def find_similar_images_cosine(given_img_path, dataset_features, dataset_image_paths, model, top_n=5):
    given_img_features = extract_features(given_img_path, model)
    if given_img_features is None:
        return [], []
    
    similarities = cosine_similarity([given_img_features], dataset_features)[0]
    similar_indices = similarities.argsort()[-top_n:][::-1]
    top_similarities = similarities[similar_indices]
    top_similar_images = [dataset_image_paths[i] for i in similar_indices]
    
    return top_similar_images, top_similarities

top_5_similar_images, top_5_similarities = find_similar_images_cosine(given_img_path, dataset_features, dataset_image_paths, model, top_n=5)

print("Top 5 similar images:")
for img_path, similarity in zip(top_5_similar_images, top_5_similarities):
    print(f"{img_path} - Similarity: {similarity * 100:.2f}%")


In [ ]:
plt.figure(figsize=(10, 5))
idx = 0
for actor_image_path, similarity in zip(top_5_similar_images, top_5_similarities):
    print(f"Displaying image: {actor_image_path}")
    img = mpimg.imread(actor_image_path)
    plt.subplot(1, 5, idx + 1)
    plt.imshow(img)
    plt.title(f"Similarity: {similarity * 100:.2f}%")
    plt.axis('off')
    idx += 1
plt.show()